# Map IDC Compounds to DrugBank

Uses https://mychem.info/

[MicroSource International Drug Collection](http://www.msdiscovery.com/intdrug.html)

Another option would be using [pubchem](https://pubchempy.readthedocs.io/en/latest/guide/substance.html#retrieving-substances) which seems to have the IDC identifiers deposited as `"Pharmakon1600-00300607"[CompleteSynonym]`.

In [1]:
import pandas
import requests

In [2]:
idc_df = pandas.read_excel(
    io="data/tamai-screen/IDC_Period_24-120.xlsx",
    sheet_name="MicroSource Discovery社311品目",
)
idc_df.head(2)


,IDNUMBER,Plate,POSITION,Name,structure,Formula,moecularl weight,saltdata,Medical Use,Reference,CAS,Status,etc
0,1506107,151126-01,A02,SULFACARBAMIDE,NaN,C7H9N3O3S,215.23208,NaN,antibacterial,Antimicrob Agents Chemother 19:82 (1981),547-44-4,synthetic,"INN, BAN"
1,210369,151126-01,A03,GALLIC ACID,NaN,C7H6O5,170.12287,NaN,"antineoplastic, astringent, antibacterial",J Chem Soc 1961:1854; Agric Biol Chem 30:617 (...,149-91-7,insect galls,"INN, NF-VII"


In [3]:
def solr_escape(text):
    """
    Escape reserved characters for pysolr queries.
    https://lucene.apache.org/solr/guide/6_6/the-standard-query-parser.html#TheStandardQueryParser-EscapingSpecialCharacters
    https://docs.mychem.info/en/latest/doc/chem_query_service.html#escaping-reserved-characters
    """
    import re
    reserved_chars = '+ - = && || > < ! ( ) { } [ ] ^ " ~ * ? : \ /'.split()
    pattern = re.compile('|'.join(map(re.escape, reserved_chars)))
    return pattern.sub(repl= lambda m: f"\\{m.group()}", string=text)


def get_mychem_query(name):
    """
    https://lucene.apache.org/solr/guide/6_6/the-standard-query-parser.html
    """
    escaped_name = solr_escape(name)
    search_fields = ["drugbank.name", "drugbank.synonyms"]
    query = " OR ".join(f'{field}:"{escaped_name}"' for field in search_fields)
    query = f"_exists_:drugbank AND ({query})"
    return query


def query_mychem(name):
    url = "https://mychem.info/v1/query"
    params = dict(
        q=get_mychem_query(name),
        fields="drugbank.name,drugbank.id,drugbank.synonyms,drugbank.formula,drugbank.weight",
        size=1,
    )
    response = requests.get(url, params)
    # solr doesn't have a whole target string match mode.
    # For example, "Gallic acid" will match "gallic acid bismuth basic salt"
    return response



In [4]:
idc_df.Name[:5]

0          SULFACARBAMIDE
1             GALLIC ACID
2                 KHELLIN
3    DEQUALINIUM CHLORIDE
4      HYDROXYTOLUIC ACID
Name: Name, dtype: object

In [5]:
rows = []
for name in idc_df.Name:
    response = query_mychem(name)
    # print(response.url)
    for hit in response.json()["hits"]:
        hit["query_compound"] = name
        rows.append(hit)
query_df = pandas.json_normalize(rows)
query_df.head(2)

,_id,_score,query_compound,drugbank._license,drugbank.formula,drugbank.id,drugbank.name,drugbank.synonyms,drugbank.weight.average,drugbank.weight.monoisotopic
0,JAONZGLTYYUPCT-UHFFFAOYSA-K,8.772476,GALLIC ACID,http://bit.ly/2PSfZTD,C7H5BiO6,DB13909,Bismuth subgallate,"[basic bismuth 3,4,5-trihydroxybenzoate, basis...",394.091,393.989010
1,GPTXWRGISTZRIO-UHFFFAOYSA-N,14.978941,CHLORQUINALDOL,http://bit.ly/2PSfZTD,C10H7Cl2NO,DB13306,Chlorquinaldol,"[2-methyl-5,7-dichloro-8-hydroxyquinoline, 5,7...",228.070,226.990469


In [6]:
query_df = query_df.reindex(columns=["query_compound", "drugbank.id", "drugbank.name", "drugbank.formula", "drugbank.weight.average"])

In [7]:
merged_df = (
    idc_df
    .reindex(columns=["IDNUMBER", "Name", "Formula", "moecularl weight"])
    .add_prefix("idc_")
    .merge(query_df, left_on="idc_Name", right_on="query_compound")
)
merged_df.head(3)

,idc_IDNUMBER,idc_Name,idc_Formula,idc_moecularl weight,query_compound,drugbank.id,drugbank.name,drugbank.formula,drugbank.weight.average
0,210369,GALLIC ACID,C7H6O5,170.12287,GALLIC ACID,DB13909,Bismuth subgallate,C7H5BiO6,394.091
1,212151,CHLORQUINALDOL,C10H7Cl2NO,228.07939,CHLORQUINALDOL,DB13306,Chlorquinaldol,C10H7Cl2NO,228.070
2,300546,BERGAPTEN,C12H8O4,216.19516,BERGAPTEN,DB12216,Bergapten,C12H8O4,216.192


In [8]:
merged_df.to_csv("data/compounds/idc-to-drugbank.tsv", sep="\t", index=False)

In [11]:
name_mismatch_df = merged_df[merged_df.idc_Name.str.lower() != merged_df["drugbank.name"].str.lower()]
name_mismatch_df.head(2)

,idc_IDNUMBER,idc_Name,idc_Formula,idc_moecularl weight,query_compound,drugbank.id,drugbank.name,drugbank.formula,drugbank.weight.average
0,210369,GALLIC ACID,C7H6O5,170.12287,GALLIC ACID,DB13909,Bismuth subgallate,C7H5BiO6,394.0910
11,1500621,AMINOPYRINE,C13H17N3O,231.29994,AMINOPYRINE,DB01424,Aminophenazone,C13H17N3O,231.2936


In [13]:
# mismatched names with different weights
name_mismatch_df[(name_mismatch_df["idc_moecularl weight"] - name_mismatch_df["drugbank.weight.average"]).abs() > 1.0]

,idc_IDNUMBER,idc_Name,idc_Formula,idc_moecularl weight,query_compound,drugbank.id,drugbank.name,drugbank.formula,drugbank.weight.average
0,210369,GALLIC ACID,C7H6O5,170.12287,GALLIC ACID,DB13909,Bismuth subgallate,C7H5BiO6,394.0910
26,1501111,PROTOPORPHYRIN IX,C34H34N4O4,562.67448,PROTOPORPHYRIN IX,DB02949,2-Acetyl-Protoporphyrin Ix,C34H34FeN4O5,634.5030
32,1501173,ACETANILIDE,C8H9NO,135.16703,ACETANILIDE,DB01297,Practolol,C14H22N2O3,266.3361
55,1506163,SELENOMETHIONINE,C5H11NO2Se,196.10892,SELENOMETHIONINE,DB09400,Selenomethionine Se-75,C5H11NO2Se,192.0710
84,1503921,CYPROTERONE,C22H27ClO3,374.91169,CYPROTERONE,DB04839,Cyproterone acetate,C24H29ClO4,416.9380
85,1503941,THIOCTIC ACID,C8H14O2S2,206.32758,THIOCTIC ACID,DB06253,Thioctic acid tromethamine,C12H25NO5S2,327.4500
100,1502248,GLUTATHIONE,C10H17N3O6S,307.32749,GLUTATHIONE,DB03310,Glutathione disulfide,C20H32N6O12S2,612.6310
131,1701028,CORTISONE,C21H28O5,360.45431,CORTISONE,DB01380,Cortisone acetate,C23H30O6,402.4807
138,1503973,2-THIOURACIL,C4H4N2OS,128.15328,2-THIOURACIL,DB00550,Propylthiouracil,C7H10N2OS,170.2320
